# Resample DEMs for plotting

In [10]:
import pathlib
import rioxarray
import geopandas
import shapely
import numpy

In [11]:
crs = 2193
base_path = pathlib.Path(r"C:\Local\data\catchments\Waikanae\caches\roughness")

In [12]:
x0=1767800; x1=1775500
y0=5469612; y1=5477770
bbox = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0,y0],[x0,y1],[x1,y1],[x1,y0]])], crs=crs)

In [14]:
dem_geofabrics = rioxarray.rioxarray.open_rasterio(base_path / "geofabrics" / "geofabric_4m_with_waterways.nc",
            masked=True,
            parse_coordinates=True,
        ).squeeze("band", drop=True)
dem_geofabrics["data_source"] = dem_geofabrics.data_source.astype(numpy.float32)
dem_geofabrics["lidar_source"] = dem_geofabrics.data_source.astype(numpy.float32)
dem_geofabrics["z"] = dem_geofabrics.z.astype(numpy.float32)
dem_geofabrics=dem_geofabrics.rio.clip(bbox.geometry)

In [16]:
dem_smart = rioxarray.rioxarray.open_rasterio(base_path / "manual_smart_geofabrics" / "dtm_final_4_5m_combined1_bridgesremoved_w_river_ocean_v2.nc",
            masked=True,
            parse_coordinates=True,
        ).squeeze("band", drop=True)
dem_smart.rio.write_transform(inplace=True)
dem_smart.rio.write_crs(crs, inplace=True)
dem_smart.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
dem_smart=dem_smart.rio.clip(bbox.geometry)
dem_smart=dem_smart.interp(x=dem_geofabrics.x, y=dem_geofabrics.y, method="linear")

# Save regridded rasters for plotting

In [19]:
dem_geofabrics.to_netcdf(base_path / "for_figures" / "geofabric_4m_with_waterways.nc", format="NETCDF4", engine="netcdf4",)
dem_smart.to_netcdf(base_path / "for_figures" / "dtm_final_4_5m_combined1_bridgesremoved_w_river_ocean_v2.nc", format="NETCDF4", engine="netcdf4",)